In [1]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn
import numpy as np
import sys
import json

/home/chao2/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv('EI_validation_input.csv',header=None)
data

,0,1,2,3,4,5
0,1.0,0.0,1.0,1.0,1.0,1.0
1,1.0,1.0,0.0,1.0,1.0,1.0
2,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,1.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,1.0,1.0
5,0.0,1.0,0.0,0.0,1.0,1.0


In [3]:
results = pd.read_csv('EI_validation_output.csv',header=None)
results

,0,1,2
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,1.0,0.0,0.0


In [4]:
print("audit_deltaEI.data") 
p=pd.read_pickle("audit_deltaEI.data")

audit_deltaEI.data


In [5]:
ctx = mx.cpu()
layer = nn.Sequential()
with layer.name_scope():
    layer.add(
        nn.Dense(6, activation="relu"),
        nn.Dropout(0.5),          # dropout to ease the overfitting problem
        nn.Dense(3)
    )
layer.load_parameters('audit_deltaEI.params',ctx)

In [6]:
predict=nd.array(data)
predict


[[1. 0. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]
 [0. 1. 0. 0. 1. 1.]]
<NDArray 6x6 @cpu(0)>

In [7]:
test_result=layer(predict).argmax(axis=1).astype(int)
print('Prediction \n')
sys.stdout.write(json.dumps(test_result.asnumpy().tolist()))

Prediction 

[2, 1, 1, 2, 1, 1]

In [8]:
audit_result=nd.array(results).argmax(axis=1).astype(int)
print('Audit results \n')
sys.stdout.write(json.dumps(audit_result.asnumpy().tolist()))

Audit results 

[1, 1, 2, 2, 0, 0]

In [9]:
comp_result=test_result-audit_result
sys.stdout.write(json.dumps(comp_result.asnumpy().tolist()))

[1, 0, -1, 0, 1, 1]